## **Efficient Programming in Python**
---
The topic of this notebook is efficient programming in Python. In real life most people when trying to solve a programming problem are less concerned with the efficiency of the solution than the actual solution. Our process is somewhat like this: 

 1. We receive the problem.
 2. We conceive a solution.
 3. We test the solution for correctness.
 4. The solution goes to production.
 5. We discover that the solution uses a lot of resources in production and/or is extremely slow.
 6. We refactor our solution to increase speed and to reduce resource usage.
 
In todays practice of agile software development I am not even sure that this is against best practice. In fact it seems almost that we should not spend too much time at point two, architecture an underated endeavour. My opinion would be to not go straight to a MVP, but to consider carefully how we built our solution. To take efficiency in both in time and space serious from the design up. Perhaps even before, at the choice of programming language. In case you program in Python, a quick way to improve the speed of your program is not to program in Python. If you move to a compiled language without garabage collection, you will increase the speed tremedously. Of course there are some risks involved, with that approach and since these notebooks are about Python we shall continue with efficiency in Python.

There are two levels of solutions to create efficient programs. At a deep level we can implement concurrency, program asynchronously or parallel. Of course, programming like that brings issues, at minimum a programmer needs to be very well versed in that technology to not make seriously costly mistakes. Though I want to discuss asynchronous programming (I find these subjects interesting), there is another manner with which to tackle the issues of inefficiency. These are higher level solutions, involving programming techniques such as:

 * optimization 
 * divide & conquer
 * greedy algorithms 
 * dynamic programming 
 * using the right data structures

These are still not always easy solutions, they need more thought, blow up your code base, and need practice. This notebook is about using these to create more efficient code. Below an example of simple direct search.

In [1]:
from typing import Any

def lin_search(lst:list[Any],element:Any)->bool:
    for v in lst:
        if v == element:
            return True
    return False

lst = [*range(1,100_000_000)]
%timeit lin_search(lst,59_674_000)

3.21 s ± 673 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Now the same problem coded up with binary search

In [2]:
def binary_search(lst:list[Any],element:Any)->bool:
    
    def search(lst:list[Any],element:Any,low:int,high:int)-> bool:
        if high == low: 
            return lst[low] == element
        mid = (high + low) // 2
        if lst[mid] == element: 
            return True
        elif lst[mid] > element:
            if low == mid: # the search has exhausted
                return False
            else:
                return search(lst,element,low,mid-1) 
        else:
            return search(lst,element,mid+1,high)
    
    if len(lst) == 0:
        return False
    else:
        return search(lst,element,0, len(lst)-1)

%timeit binary_search(lst,59_674_000)

10.2 µs ± 2.22 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


#### **Analysis the shortest introduction ever**
As you can see we have moved from second to microsecond, that is a million times faster. Binary search is not even the fastest algorithm possible, for instance, merge sort is faster. I am not that interested in the "fastest" you should assume Python has implemented the fasted algorithm for general circumstances. If Timsort does not meet your needs, you will know more than I do both about programming and mathematics, you should write notebooks :-). 

What I am interested in is to show you several programming techniques with examples, and give you an insight why these are more efficient. To do that I have to introduce you very quickly to the technique used to analyze algorithms. Firstly when we analyze algorithms we are foremostly interested in the running time of the algorithm, secondary is the space in memory the algorithm requires. I will use `%timeit` just to have an experimental way if showing code is faster or slower. Experimenting, though fun, is not the way to analyze algorithms. After all, speed would definitely be influenced on the hardware (mine is old and not that fast, yet beautiful and loved) but also on the input size. Waiting however, is boring, so we cannot use enormous input sizes.

Luckliy enough for us some people already thought of the tools you need to analyze algorithms based upon on a high level description of the algorithm. To do that we first consider all the things we think are primitive operations. We will define the following as primitive operations:

 * Assigning an identifier to an object
 * Determining the object associated with an identifier
 * Performing an arithmetic operation (e.g., adding two numbers)
 * Comparing two numbers
 * Accessing a single element of a Python list by index
 * Calling a function / method (excluding operations executed within the function)
 * Returning from a function / method

All these primitive operations are exexcuted in constant time. With this in mind we can straight away see why binary search is so much faster than linear search. The latter needs to do nearly sixty million constant operations. The former basically divides the list in two, sees if the element wanted is bigger or smaller than the middle, moves over to whatever is the case and repeats the operation until the number is found or there are no more numbers.

{ `binary_search([50,000,000, ..., 100,000,000], 59,674,000)` }
{ `binary_search([50,000,000, ..., 75,000,000], 59,674,000)` }
{ `binary_search([50,000,000, ..., 62,500,000], 59,674,000)` }
{ `binary_search([56,200,000, ..., 62,500,000], 59,674,000)` }
{ `binary_search([593,000,000, ..., 62,500,000], 59,674,000)` }
 
Within 5 runs of the algorithm binary search is close to the target. Linear search still will have fifty-nine million and a bit steps to go.   

When we analyze algorithms it is easiest to assume worst case scenarios. It is often easy to identify a worst case scenario. Furthermore if you design algorithms to perform good (as good as can be achieved) for worst case, than obviously they will perform better in better scenarios. Big O notation gives us the worst case scenario for the time an algorithm needs to compute a computation.

It is typical to use seven functions to describe speed of algorithms.

 1. f(n) = c $\rightarrow O(1)$ constant time is important because it signals the amount of steps needed to do a basic operation, e.g., add two numbers together.
 2. $O(log(n))$ or  $O(log_2(n))$ logarithmic time. We quite often see logarithmic function creeping up in analysis. Yet 
 


In [5]:
59674000-59300000

374000

#### Code comment
Let's consider what happens.We receive a list and keep looking through the list and comparing elements.
#### in this scenario we have to compare all elements in the list to element n times.With being the length of the list. 
#### We say this algorithm takes O(n) times to complete. This is worst case scenario of course if we were looking for 1 we would have found it immidiately O(1)
#### 
#### From fastest to slowest in big O:
#### $O(1)$ a.k.a constant time
#### $O(log(n))$
#### $O(n)$
#### $O(log(n) \times n) $
#### $O(n^2)$ a.k.a polynomial time
#### $O(2^n)$ a.k.a exponential time
#### We can quite obviously do this faster with a binary search

#### As we can see linear search takes roughly 3 milliseconds and binary search takes roughly a thousand times less with 5 micro seconds
#### We can also see that binary search is much the less straight forward code
#### Programming efficient fast code is not easy, it is often a trade off between writing simple but slow code for more complex but faster code 
#### This is important in real time systems for instance stock trading or software in cars or airplanes
#### Most of this type of code is not written in Python, but languages like Ada and Rust
#### In Python especially when working with large data you should know how to program efficiently to a degree
#### This notebook is about the type of problems that can be solved efficiently

## **Croc and Pinky are going to rob the butcher (Optimization)**
#### There are great snackies inside the butcher which need to rescued and eaten. Croc has considered it and gave the following values to the goodies in the butcher shop:
#### Tenderloin - 175
#### Shoulder of lamb - 90
#### Porkbelly - 20
#### Surloin - 50
#### Sausage - 10 
#### Cote du Boeuf - 200
#### Pinky being the sensible one has noted how much there is of each item and noted the weights:
#### Tenderloin - 10
#### Shoulder of lamb - 9
#### Porkbelly - 4
#### Surloin - 2
#### Sausage - 1 
#### Cote du Boeuf - 20
#### Now they have a conundrum how the get the most value without exceeding the carrying limit of their innocent little baby arms and mouths (20kg the constraint)
#### As you explained to Croc you can tackle this optimization problem in two ways, you can opt for the optimal solution, or you can go for the greedy one. To Croc this was nuff said, the greedy solution it had to be as was very greedy indeed and so was Pinky.
#### Let's help this salty and his dragon girlfriend!

In [ ]:
constraint = 20

from dataclasses import dataclass

@dataclass
class Item:
    name:str
    value:int
    weight:float
    
def build_items()->list[Item]:
    meat = ['Tenderloin', 'Shoulder of lamb', 'Porkbelly', 'Surloin', 'Sausage','Cote du Boeuf']
    values = [175,90,20,50,10,200]
    weights = [10,9,4,25,1,20]
    items = []
    for idx in range(len(meat)):
        items.append(Item(meat[idx], values[idx], weights[idx]))
    return items

items = build_items()
 
# objective functions    
def value(item:Item)->int:
    return item.value

def weight_inverse(item:Item)->float:
    return 1 / item.weight

def density(item:Item)->float:
    return item.value / item.weight

# The greedy solution
def greedy(items:list[Item], constraint:int, key_func:object)->tuple[list[Item],int]:
    items = sorted(items, key=key_func, reverse=True)
    res = []
    total_value, total_weight= 0.0,0.0
    for item in items:
        if total_weight + item.weight <= constraint:
            res.append(item)
            total_weight += item.weight
            total_value += item.value
    return res,total_value

In [ ]:
greedy(items, constraint, value)

#### Croc surely will be happy but Pinky will be mighty ticked off as Croc never shares his cote du boeuf
#### She wants another objective function

In [ ]:
greedy(items, constraint, weight_inverse)

#### Now both Croc and Pinky are angry this is simply not acceptable loot we need another objective function!

In [ ]:
greedy(items, constraint, density)

#### it seems the best optimizing function is a ratio of weight/value and not the value of the item itself.

## **An optimal solution**
#### Unfortunately when Croc learned he couldn't take the cote du boeuf he threw an epic hissy fit and he and is girlfriend Pinky where caught by the police, again...
#### To prevent any further episodes he now wants an optimal solution, the best possible in all circumstances
#### A formallization of the problem is:
#### item is the pair (value,weight)
#### they can take no more of the pairs than the constraint, of 20 kg
#### There is a set of available items snackies, a vector I of length n represents snackies
#### There is a vector V of length n with binary values, if `V[i]==1` than the snacky will be taken `V[i]==0` the snacky will be left behind, but under loud protest.
#### Now the only thing we have to do is find vector V that maximizes $\sum_{i=0}^{n-1} V[i]*I[i].value$ 
#### Subject to the constraint $\sum_{i=0}^{n-1} V[i]*I[i].weight \le constraint$  

#### Our solution is going to be in three steps
#### The first step to create a powerset of the snackies set (all possible subsets including snackies itself and the empty set)

In [ ]:
# The itertools library is very rich with useful functions, this one prevents us from having to write a powerset function our selves
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

pset = list(powerset(items))[1:]
len(pset)

#### In the second step we remove all sets that exceed the weight constraint

In [ ]:
def rm_excess_weight(pset:list[tuple[Item]], constraint:int)->list[tuple[Item]]:
    res = []
    for items in pset:
        weight = 0.0
        for item in items:
            weight += item.weight
        if weight <= constraint:
            res.append(items)
    return res

pset = rm_excess_weight(pset,constraint)
len(pset)

#### As third and final step we only need to select that set with best overall value

In [ ]:
def snackies(pset:list[tuple[Item]])->tuple[Item]:
    snacks = ()
    best_value = 0.0
    for items in pset:
        value = 0.0
        for item in items:
            value += item.value
        if value > best_value:
            snacks = items
            best_value = value
    return snacks

In [ ]:
snackies(pset)

#### Unfortunately for Croc, it does make sense to leave the Cote du Boeuf behind. The best value is achieved by taking the above items. 
#### Fortunately dragons can spit fire and Croc heeled and the butcher was succesfully robbed

## **Greedy**
#### Here it ends for Croc and Pinky but we should consider the above code.
#### We wil work backwards and start considering snackies
#### We have two iterables, both we would have to traverse completely in the worst case scenario. $O(n^2)$
#### Now for rm_excess_weight this has only one iterable we need to traverse at worst this will take $O(n)$ 
#### Now we get to the kicker powerset here it is reasonably efficient

In [ ]:
l = set(powerset([1,2,3]))
l

In [ ]:
len(l)

In [ ]:
k = set(powerset([1,2,3,4]))
len(k)

#### I am sure you feel it coming but let's have a look on the powerset of a set with 5 elements

In [ ]:
j = set(powerset([*range(1,6)]))
len(j)

#### Powerset is exponential in growth, as the input size grows the output size grows exponential $O(2^n)$
#### Unfortunately there is nothing we can do. Taking the powerset of a set is always exponential relative to the size of the set. 
#### So the actual algorithmic complexity of the whole code is $O(2^n) + O(n^2) + O(n)= O(2^n)$
#### This might suprise you but as n get's bigger $O(2^n)$ quickly dwarfs $O(n^2)$ so the latter two become insignificant, so we only count the largest  

In [ ]:
2**100, 100**2

#### The point of the story comes back to Herbert Simon who said:
#### Models making decisions that are good enough rather than laboriously calculating the optimal result are a better description of human or Croc behaviour. We booth like to do things greedy, just accepting to take that result that is satisfying. Herbert Simon won a Nobel prize for this insight.

## **Optimization**
#### Programming solutions to problems often entails recognizing the pattern in the problem and applying specific programming styles to create a solution. 
#### Optimization is a very common problem for the developer to solve. For instance how much shoes a factory could produce given the material in stock, or what is the itinerary of travelling salesman?
#### An optimization problem exists of two parts:
#### 1 The objective function that is to be minimised or maximised. For instance the cost of a plane ticket between two cities when doing a search online, it needs to be the lowest possible in general.
#### 2 A set of constraints that must be honored. For instance a flight from Amsterdam to Liverpool should not take 6 hours and two flight changes. This set of constraints might include the empty set. 
#### These types of problems are very common. They also tend to be problems that can be formulated in a simple manner that lead to naturally computational solutions. They are problems that occur a lot in data intensive applications.
#### These problems are in general reducable to well known problems (they follow the same pattern)
#### These problems can be solved with exhaustive enumerations algorithms, but because of the sheer amount of computation involved, more often than not greedy algorithms are used that deliver a fast sub-optimal but acceptable solution. 
#### Quite often you will find that these problems can be solved recursively.

In [ ]:
def product(numbers:list[int])->int:
    return recursion(numbers,1)

def recursion(numbers:list[int], acc:int)->int:
    if len(numbers) == 1:
        return acc*numbers[0]
    acc = numbers.pop()*acc
    return recursion(numbers,acc) 
product([*range(1,5),5]) 

#### unfortunately Python is not the best at recursion it has a very limited recursion depth

In [ ]:
import sys
sys.getrecursionlimit()

#### We can enlarge the recursion limit to say 100_000

In [ ]:
sys.setrecursionlimit(100_000)

#### However while 100_000 might seem like a big number it really isn't.
#### In Python recursion is also not the manner with which Python handles repitition, it uses iteration
#### However for problems where recursion is natural, like product, sum or factorial or so, programming without recursion requires additional coding techniques 
#### Techniques like memoization and tabulation that create a solution optimized for Python 

## **Tabulation example**

In [ ]:
def tab_product(ns:list[int])->int:
    l = len(ns)
    tab = [None]*(l+1)
    tab[0] = 1
    for idx in range(1,l+1):
        tab[idx] = ns[idx-1] * tab[idx-1]
    return tab[l]
tab_product([1,2,3,4,5])

## **Memoization example**

In [ ]:
memoization = {}

def mem_product(ns:list[int])->int:
    memoization[0] = ns[0]
    for idx in range(1,len(ns)):
        memoization[idx] = ns[idx]*memoization[idx-1]
    return memoization[len(ns)-1]
mem_product([1,2,3,4,5])    

In [ ]:
ls = [i for i in range(10,10_000,10)]
%time mem_product(ls)

#### In general the method is as follows, first follow the natural pattern and create a solution using recursion
#### Once you have developed that solution we can optimize it for use with Python by reimplementing the solution using tabulation or memoisation